In [7]:
import sys
print(sys.executable)

c:\Users\RISHIVEER YADAV\AppData\Local\Programs\Python\Python39\python.exe


# 3Q) KNN and ANN
In this question we are asked to implement KNN and ANN from scratch given input embeddings

## 3.1) KNN
We are asked to implement KNN from scratch, the inputs are a set of embeddings. Given a query, k and a distance metric, it should return the indices of the k nearest neighbours from the database.

In [8]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

def euclidean_distance(a, b):
    a_sq = np.sum(a**2, axis=1, keepdims=True)
    b_sq = np.sum(b**2, axis=1)
    ab = np.dot(a, b.T)
    return np.sqrt(a_sq - 2 * ab + b_sq)

def cosine_distance(a, b):
    a_norm = a / np.linalg.norm(a, axis=1, keepdims=True)
    b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
    return 1 - np.dot(a_norm, b_norm.T)

In [9]:
def knn(database, query, k, distance_metric):
    indices = []
    if distance_metric == 'euclidean':
        distance = euclidean_distance(query, database)
    elif distance_metric == 'cosine':
        distance = cosine_distance(query, database)
    else:
        raise ValueError("Distance Metric not supported. Use only Euclidean or Cosine distances")
    indices.append(np.argsort(distance, axis=1)[:, :k])
    return np.vstack(indices)

In [12]:
def classify(database, labels, query, k, distance_metric):
    neighbour_indices = knn(database, query, k, distance_metric)
    neighbour_labels = labels[neighbour_indices]
    predictions = np.array([np.bincount(neighbour).argmax() for neighbour in neighbour_labels])
    return predictions

def mean_reciprocal_rank(ranked_indices, query_labels, database_labels):
    mrr_values = []
    for i, (ranks, true_label) in enumerate(zip(ranked_indices, query_labels)):
        # Find the first occurrence of the correct class
        correct_indices = np.where(database_labels[ranks] == true_label)[0]
        if len(correct_indices) > 0:
            mrr_values.append(1.0 / (correct_indices[0] + 1))
        else:
            mrr_values.append(0.0)
    return np.mean(mrr_values)

def precision_at_k(ranked_indices, query_labels, database_labels, k=100):
    precisions = []
    for ranks, true_label in zip(ranked_indices, query_labels):
        retrieved_labels = database_labels[ranks[:k]]
        precisions.append(np.sum(retrieved_labels == true_label) / k)
    return np.mean(precisions)

def hit_rate(ranked_indices, query_labels, database_labels):
    hits = []
    for ranks, true_label in zip(ranked_indices, query_labels):
        retrieved_labels = database_labels[ranks]
        hits.append(1 if true_label in retrieved_labels else 0)
    return np.mean(hits)

In [13]:
try:
    train_embeddings = torch.load('./SMAI A1/train_embeddings.pth',map_location=torch.device('cpu')).numpy()
    test_embeddings = torch.load('./SMAI A1/test_embeddings.pth',map_location=torch.device('cpu')).numpy()
    text_embeddings = torch.load('./SMAI A1/text_embedding.pth',map_location=torch.device('cpu')).numpy()
    train_labels = torch.load('./SMAI A1/train_labels.pth',map_location=torch.device('cpu')).numpy()
    test_labels = torch.load('./SMAI A1/test_labels.pth',map_location=torch.device('cpu')).numpy()

    train_embeddings /= np.linalg.norm(train_embeddings, axis=1, keepdims=True)
    test_embeddings /= np.linalg.norm(test_embeddings, axis=1, keepdims=True)
    text_embeddings /= np.linalg.norm(text_embeddings, axis=1, keepdims=True)
except Exception as e:
    print('Error has occured:', e)

In [15]:
# Classification using train embeddings
for k in [1, 5, 10]:
    pred_euc = classify(train_embeddings, train_labels, test_embeddings, k, distance_metric='euclidean')
    pred_cos = classify(train_embeddings, train_labels, test_embeddings, k, distance_metric='cosine')
    print(f'k={k} Euclidean Accuracy:', accuracy_score(test_labels, pred_euc))
    print(f'k={k} Cosine Accuracy:', accuracy_score(test_labels, pred_cos))

# Classification using text embeddings
pred_text = classify(text_embeddings, np.arange(10), test_embeddings, k=1, distance_metric='cosine')
print('Text-based Classification Accuracy:', accuracy_score(test_labels, pred_text))

# Text-to-Image Retrieval
text_labels = np.arange(10)
neighbours_text_to_image = knn(train_embeddings, text_embeddings, k=100, distance_metric='cosine')
print('\nText-to-Image Retrieval Metrics:')
print('MRR:', mean_reciprocal_rank(neighbours_text_to_image, text_labels, train_labels))
print('Precision@100:', precision_at_k(neighbours_text_to_image, text_labels, train_labels))
print('Hit Rate:', hit_rate(neighbours_text_to_image, text_labels, train_labels))

# Image-to-Image Retrieval
neighbours_image_to_image = knn(train_embeddings, test_embeddings, k=100, distance_metric='cosine')
print('\nImage-to-Image Retrieval Metrics:')
print('MRR:', mean_reciprocal_rank(neighbours_image_to_image, test_labels, train_labels))
print('Precision@100:', precision_at_k(neighbours_image_to_image, test_labels, train_labels))
print('Hit Rate:', hit_rate(neighbours_image_to_image, test_labels, train_labels))

k=1 Euclidean Accuracy: 0.9048
k=1 Cosine Accuracy: 0.9048
k=5 Euclidean Accuracy: 0.9182
k=5 Cosine Accuracy: 0.9182
k=10 Euclidean Accuracy: 0.9194
k=10 Cosine Accuracy: 0.9194
Text-based Classification Accuracy: 0.8781

Text-to-Image Retrieval Metrics:
MRR: 1.0
Precision@100: 0.974
Hit Rate: 1.0

Image-to-Image Retrieval Metrics:
MRR: 0.934796151331504
Precision@100: 0.841083
Hit Rate: 0.9996
